In [57]:
import torch
import gpytorch
import pinot
import math


from torch.optim.lr_scheduler import MultiStepLR


In [58]:
layer = pinot.representation.dgl_legacy.gn()
net = pinot.representation.Sequential(
    layer,
    [32, 'tanh', 32, 'tanh', 32, 'tanh'])

In [59]:
ds = pinot.data.esol()
ds = pinot.data.utils.batch(ds, 8)
ds_tr, ds_te = pinot.data.utils.split(ds, [4, 1])

In [68]:
class ExactGP(gpytorch.models.ExactGP):
    def __init__(self, x_tr, y_tr, likelihood):
        super(ExactGPModel, self).__init__(x_tr, y_tr, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        x = self.net(x)
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
class DeepGraphGP(torch.nn.Module):
    def __init__(self):
        self.net = pinot.representation.Sequential(
            layer,
            [32, 'tanh', 32, 'tanh', 32, 'tanh'])
        
        self.gp = ExactGP()

In [67]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel([g], [y], likelihood)


RuntimeError: Train inputs must be a tensor, or a list/tuple of tensors

In [55]:
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
# scheduler = MultiStepLR(optimizer, milestones=[0.5 * n_epochs, 0.75 * n_epochs], gamma=0.1)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=len(ds_tr))

In [56]:
for g, y in ds_tr:
    optimizer.zero_grad()
    y_hat = model(g)
    print(y_hat)
    loss = -mll(y_hat, y)
    loss.backward()
    optimizer.step()

torch.Size([8, 32])
torch.Size([32, 8, 1])
MultitaskMultivariateNormal(loc: torch.Size([256]))


RuntimeError: shape '[8, 8, 32]' is invalid for input of size 256